# Full Object Based Image Analysis (OBIA)

1. Import and preprocess images (remove outliers and rescale intensity)
2. Segment images (parameters per stratum)
3. Vectorize segments, then extract features for each segment
4. Train classification model and output results (and scaler)

Now parallelized :sunglasses:

In [1]:
import os
import subprocess

from functools import partial
from multiprocessing import Pool

import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio

from joblib import dump
from rasterio.features import shapes
from sklearn import svm
from sklearn import preprocessing
from skimage.exposure import rescale_intensity
from skimage.segmentation import felzenszwalb
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

from fetex_functions import get_features_cols, extract_feature_df
from segmentation_functions import get_fid, read_stratum_images, tiff_export, gpkg_export

pd.options.display.max_columns = 500

In [2]:
# Number of cores to use for multiprocessing
nc = os.cpu_count()

# Samples to ignore (for now)
blacklist = [str(x) for x in [10, 17, 22]]

In [3]:
def rescale_band(band, old_min, old_max, new_min, new_max):
    return ((band - old_min)/old_max) * (new_max - new_min)

def reject_outliers(data, m, no_data):
    data_mean = np.mean(data[data != no_data])
    data_std = np.std(data[data != no_data])
    max_val = data_mean + m * data_std
    min_val = data_mean - m * data_std
    
    result = np.copy(data)
    result[result > max_val] = result[result < max_val].max() # Assign next highest existent val
    result[result < min_val] = result[result > min_val].min() # Same w/ lowest for min

    return result

def process_band(band, m=5, no_data=0, out_type='uint8', out_range=(0, 255), elev_band=False, elev_bounds=(0,25)):
    min_elev, max_elev = elev_bounds
    min_out, max_out = out_range
    if elev_band:
        # Trust min but not max
        band[band < min_elev] = min_elev
        band[band > max_elev] = band[(band >= min_elev) & (band <= max_elev)].mean()
        
        # Normalize to bounds (min_elev -> min_out, max_elev -> max_out)
        band = rescale_band(band, min_elev, max_elev, min_out, max_out)
        return band.astype(out_type)
    else:
        return rescale_intensity(
            reject_outliers(band, m, no_data),
            out_range=out_type
        ).astype(out_type)
    
def rescale_clean_img(name, img, elev_band=4, debug=True):
    # Handle super-weird data
    if debug:
        print(f'Working on {name}')
    img = img['img']
    img[img < 0 ] = 0

    # Then just normal-weird
    # We'll scale each band independently
    # Last band is LiDAR so treatment is different
    processed_bands = []
    for band in range(0, img.shape[2]):
        is_elev_band = True if band == elev_band else False
        processed_bands.append(process_band(img[:,:,band], elev_band=is_elev_band))
            
    return (name, np.dstack(processed_bands))

## Multiprocessing wrappers

def read_imgs_mult(stratum):
    return read_stratum_images(base_path='../data/imgs_final/', stratum=stratum, blacklist=blacklist)

def felz_mult(name, img_8, scale, sigma, min_size, debug=False, bands=None):
    if bands:
        s = felzenszwalb(img_8[:,:,bands], scale=scale, sigma=sigma, min_size=min_size)
    else:
        s = felzenszwalb(img_8, scale=scale, sigma=sigma, min_size=min_size)
    if debug:
        print(f"Img: {name}. Number of segments: {len(np.unique(s))}")
    
    return name, s

# Exports

def tiff_export_mult(sample_name, sample_segments):
    tiff_export(
        segment=sample_segments,
        props=imgs[sample_name]['props'],
        output_name=os.path.join(segmentation_path, sample_name),
        debug=True
    )

def gpkg_export_mult(sample_name, sample_segments):
    gpkg_export(
        segment=sample_segments,
        props=imgs[sample_name]['props'],
        output_name=os.path.join(segmentation_path, sample_name.replace('.tif', '.gpkg')),
        debug=True
    )

# Feature extraction

def fetex_multi(sample_name, debug=True):
    if debug:
        print(f'------ Working on {sample_name} ------')
    img = imgs_8[sample_name]
    seg = segments[sample_name]
    cols = get_features_cols(img.shape[2])
    
    return sample_name, extract_feature_df(img, seg, cols, debug=False)

## Reading and scaling to uint8 for all bands

In [4]:
strata = {
    'residential': (100, 1, 1000),
    'urbanreg': (70, 1, 800),
    'urbanirreg': (60, 1, 600),
    'rural': (50, 1, 600),
    'shanty':  (50, 1, 600)
}

In [5]:
with Pool(nc) as pool:
    imgs_list = pool.map(read_imgs_mult, strata.keys())
    # Reduce array to dict
    imgs = {k:v for d in imgs_list for k, v in d.items()}
    print('Finished reading. Scaling...')
    imgs_8 = {k:v for k,v in pool.starmap(rescale_clean_img, imgs.items())}

Finished reading. Scaling...
Working on sample_13_residential.tif
Working on sample_6_residential.tif
Working on sample_0_residential.tif
Working on sample_23_residential.tif
Working on sample_8_residential.tif
Working on sample_11_urbanreg.tif
Working on sample_20_urbanreg.tif
Working on sample_4_urbanreg.tif
Working on sample_1_urbanreg.tif
Working on sample_7_urbanreg.tif
Working on sample_14_urbanreg.tif
Working on sample_12_urbanreg.tif
Working on sample_21_urbanirreg.tif
Working on sample_15_urbanirreg.tif


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/exposure/exposure.py:351: RuntimeWarning: invalid value encountered in true_divide
  image = (image - imin) / float(imax - imin)


Working on sample_18_urbanirreg.tif
Working on sample_16_urbanirreg.tif


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/exposure/exposure.py:351: RuntimeWarning: invalid value encountered in true_divide
  image = (image - imin) / float(imax - imin)


Working on sample_5_rural.tif
Working on sample_19_rural.tif
Working on sample_3_shanty.tif
Working on sample_9_shanty.tif
Working on sample_2_shanty.tif


## Segmenting

We'll prepare an iterable for the `felz_mult` parallelized function containing the image name,
the image itself and the segmentation parameters for its stratum.

In [ ]:
segm_iterable = [(k,v,*strata[k.split('_')[-1].replace('.tif', '')], True, [0,1,2,4]) for k,v in imgs_8.items()]

In [ ]:
with Pool(nc) as pool:
    segments = {k:v for k,v in pool.starmap(felz_mult, segm_iterable)}

In [7]:
segmentation_path = "../data/imgs_segm/"
if not os.path.exists(segmentation_path):
    os.mkdir(segmentation_path)

In [ ]:
with Pool(nc) as pool:
    pool.starmap(tiff_export_mult, segments.items())
    pool.starmap(gpkg_export_mult, segments.items())

Uncomment and run this cell to **load segmented images**

In [8]:
segments_list = [read_stratum_images(x, segmentation_path) for x in strata.keys()]
segments = {k: v['img'][:,:,0] for d in segments_list for k, v in d.items()}

#### Add synthetic bands

In [9]:
for name, img in imgs_8.items():
    b7 = rescale_intensity(1000 + img[:,:,1] - img[:,:,0], out_range=np.uint8).astype(np.uint8)
    b8 = rescale_intensity(1000 + img[:,:,2] - img[:,:,1], out_range=np.uint8).astype(np.uint8)
    b9 = rescale_intensity(1000 + img[:,:,0] - img[:,:,2], out_range=np.uint8).astype(np.uint8)
    imgs_8[name] = np.dstack((img, b7, b8, b9))

#### Reconvert height

We resampled the height band to the full `np.uint8` interval `(0, 255)` for the segmentation but we need to now keep those values as actual meters. Later, in the classification, these values will actually be normalized again, but we need the originals for the MBT classification

In [25]:
for name, img in imgs_8.items():
    imgs_8[name][:,:,4] = rescale_band(imgs_8[name][:,:,4], 0, 255, 0, 25)

## Feature extraction

In [26]:
with Pool(nc) as pool:
    fdfs = {k:v for k,v in pool.map(fetex_multi, segments.keys())}

------ Working on sample_0_residential.tif ------
------ Working on sample_13_residential.tif ------
------ Working on sample_8_residential.tif ------
------ Working on sample_20_urbanreg.tif ------
630 segments to process
758 segments to process
643 segments to process
739 segments to process
------ Working on sample_11_urbanreg.tif ------
685 segments to process
------ Working on sample_4_urbanreg.tif ------
604 segments to process
------ Working on sample_23_residential.tif ------
591 segments to process
------ Working on sample_6_residential.tif ------
550 segments to process
------ Working on sample_1_urbanreg.tif ------
637 segments to process
------ Working on sample_14_urbanreg.tif ------
657 segments to process
------ Working on sample_21_urbanirreg.tif ------
920 segments to process
------ Working on sample_18_urbanirreg.tif ------
1388 segments to process
------ Working on sample_7_urbanreg.tif ------
713 segments to process
------ Working on sample_12_urbanreg.tif ------
72

Make ID unique to each sample by prefixing the FID

In [27]:
for sample, df in fdfs.items():
    df['id'] = df['id'].apply(lambda x: f'{get_fid(sample)}_{x}')

fdf = pd.concat(fdfs).reset_index()
fdf.drop(axis=1, columns=['level_0', 'level_1', 'index'], inplace=True)

## Load GCPs

In [28]:
gcps_path = os.path.abspath('../data/gcps/')
gcps = {os.path.basename(k).replace('.gpkg', ''):gpd.read_file(k) 
        for k in [os.path.join(gcps_path, x) 
                  for x in os.listdir(gcps_path) if x.endswith('gpkg')]}
    
gcps = pd.concat(gcps).reset_index()
gcps.rename(index=str, columns={'level_0': 'class', 'level_1': 'idx'}, inplace=True)

gcps.loc[gcps['class'].isin(['red_tin', 'blue_tin']), 'class'] = 'tin'

In [30]:
gcps.groupby('class')['idx'].count()

class
concrete      335
pavement      226
shadow        169
tin           397
vegetation    172
Name: idx, dtype: int64

Use this line only for LiDAR segmentation

In [31]:
# gcps = gcps.loc[gcps['class'] != 'shadow'].copy()

## Load segments GDF, merge to feature DF and get class from GCPs

In [32]:
gdfs = {x:gpd.read_file(os.path.join(segmentation_path, x)) 
        for x in os.listdir(segmentation_path)
        if x.endswith('.gpkg')}

for k, df in gdfs.items():
    fid = k.split('_')[1]
    df['segment_id'] = df['segment_id'].apply(lambda x: f'{fid}_{x}')

gdf = pd.concat(gdfs, ignore_index=True)

gdf['area'] = gdf['geometry'].area
gdf = gdf.loc[gdf.groupby('segment_id')['area'].idxmax(), :]

gdf = pd.merge(gdf, fdf, left_on='segment_id', right_on='id')

gdf = gpd.sjoin(gdf, gcps, how='left', op='contains')

In [33]:
gdf['class_right'].value_counts()

tin           346
concrete      290
pavement      198
vegetation    145
shadow        142
Name: class_right, dtype: int64

## Classify and hope for the best!

In [36]:
# Drop synthetic bands
# gdf.drop(axis=1, columns=[c for c in gdf.columns if c.endswith(('_7', '_8', '_9'))], inplace=True)

In [36]:
cols_to_drop = ['segment_id', 'area_x', 'class_left', 'geometry', 'id', 'index_right', 'idx']

df = gdf[gdf['class_right'].notnull()].copy()
df.drop(axis=1, columns=cols_to_drop, inplace=True)

y = df['class_right']
x = df.drop(['class_right'], axis=1).astype(np.float64)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

scaler = preprocessing.StandardScaler().fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

Test with SVM (not great)

In [28]:
selector = SelectKBest(k = 15)
selected_features = selector.fit_transform(x_train, y_train)
mask = selector.get_support() #list of booleans
new_features = [] # The list of your K best features

for bool, feature in zip(mask, x_train.columns):
    if bool:
        new_features.append(feature)

# Fit model
x_train_fs = pd.DataFrame(selector.fit_transform(x_train, y_train),
                      columns=new_features)
# pipeline = make_pipeline(preprocessing.StandardScaler(),
#                          svm.SVC())
clf = svm.LinearSVC(max_iter=100000)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))
print('Confusion matrix:')
print(confusion_matrix(y_test, y_pred))
print('Selected features:')
print(new_features)

              precision    recall  f1-score   support

    concrete       0.69      0.47      0.56        51
    pavement       0.60      0.15      0.24        39
      shadow       0.24      0.81      0.38        26
         tin       0.56      0.43      0.49        63
  vegetation       0.62      0.62      0.62        37

   micro avg       0.47      0.47      0.47       216
   macro avg       0.54      0.50      0.46       216
weighted avg       0.57      0.47      0.47       216

Confusion matrix:
[[24  1  8 13  5]
 [ 0  6 31  1  1]
 [ 1  0 21  2  2]
 [10  3 17 27  6]
 [ 0  0  9  5 23]]
Selected features:
['compacity_index', 'mean_1', 'mean_2', 'mean_3', 'mean_5', 'median_1', 'median_2', 'median_3', 'median_4', 'median_5', 'mode_1', 'mode_2', 'mode_3', 'mode_4', 'mode_5']


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [37]:
pipeline = make_pipeline(preprocessing.StandardScaler(),
                         RandomForestClassifier(n_estimators=100))
hyperparameters = {
    'randomforestclassifier__max_depth': [None, 5, 3, 1], 
    'randomforestclassifier__max_features': ['auto', 'sqrt', 'log2'],
    'randomforestclassifier__min_samples_leaf': [1, 5, 20, 100]
}
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
 
# Fit and tune model
clf.fit(x_train_scaled, y_train)

print(clf.best_params_)
print(clf.refit)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'randomforestclassifier__max_depth': None, 'randomforestclassifier__max_features': 'sqrt', 'randomforestclassifier__min_samples_leaf': 1}
True


In [52]:
y_pred = clf.predict(x_test_scaled)
print(f'Cohen-Kappa score: {cohen_kappa_score(y_test, y_pred)}\n')
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Cohen-Kappa score: 0.936736363171617

              precision    recall  f1-score   support

    concrete       0.99      0.99      0.99        69
    pavement       0.98      1.00      0.99        45
      shadow       0.86      0.92      0.89        26
         tin       0.93      0.96      0.95        56
  vegetation       0.96      0.79      0.87        29

   micro avg       0.95      0.95      0.95       225
   macro avg       0.94      0.93      0.94       225
weighted avg       0.95      0.95      0.95       225

[[68  0  0  1  0]
 [ 0 45  0  0  0]
 [ 0  0 24  1  1]
 [ 1  1  0 54  0]
 [ 0  0  4  2 23]]


In [41]:
dump(scaler, '../data/results/scalers/rgb_elev_segm.joblib')
dump(clf, '../data/results/models/rgb_elev_segm.joblib')

['../data/results/scalers/rgb_elev_segm.joblib']

We'll have to select columns and scale to get prediction

In [40]:
df_reclassed = gdf.drop(axis=1, columns=cols_to_drop + ['class_right']).copy()
scaled_array = scaler.transform(df_reclassed)

df_reclassed = pd.DataFrame(scaled_array, index=df_reclassed.index, columns=df_reclassed.columns)
df_reclassed['class_right'] = clf.predict(df_reclassed)

# Join prediction to GDF, then convert to UTM and prepare fields for MBT classification
gdf_to_save = gdf.drop(
    axis=1,
    columns=['class_right', 'class_left', 'area_x', 'area_y', 'id', 'index_right']
).join(df_reclassed['class_right']).copy()
gdf_to_save = gdf_to_save.to_crs({'init': 'epsg:32618'})
gdf_to_save['area'] = gdf_to_save.geometry.area
gdf_to_save['stories'] = gdf_to_save.mean_5 / 3
gdf_to_save['utm_x'] = gdf_to_save.geometry.centroid.x
gdf_to_save['utm_y'] = gdf_to_save.geometry.centroid.y

gdf_to_save.to_file('../data/results/rgb_elev_segm.gpkg', driver='GPKG')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


#### Notes

Feature selection:

['compacity_index', 'mean_1', 'mean_2', 'mean_3', 'mean_5', 'median_1', 'median_2', 'median_3', 'median_5', 'mode_1', 'mode_2', 'mode_3', 'mode_5', 'ASM_5', 'ASM_6']

Takeaway: Our assumptions have been mostly correct. Shape is important, as well as RGB + elevation, and the texture of elevation and intensity. SWIR seems to play no role.